## Import subtitles_lines data from a pickle object

In [14]:
import pickle

with open('./results/subtitles_lines.pkl', 'rb') as f:
    subtitles_lines = pickle.load(f)

In [16]:
%time
from nltk.tokenize import word_tokenize

sentences = []
for subtitle_lines in subtitles_lines:
    # first three sentences and the last one are not to be used
    for line in subtitle_lines[3:-1]:
        sentences.append(word_tokenize(line))

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [17]:
len(sentences)

112094

## Word2Vec model

In [18]:
from gensim.models import Word2Vec
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

In [19]:
# example
# from nltk.corpus import movie_reviews
# sentences = [list(s) for s in movie_reviews.sents()]

In [20]:
%%time
model = Word2Vec(sentences)

CPU times: user 3.25 s, sys: 9.27 ms, total: 3.26 s
Wall time: 1.26 s


In [21]:
model.init_sims(replace=True)

In [22]:
model.most_similar('Ross')

[('Chandler', 0.8937354683876038),
 ('Joey', 0.8852862119674683),
 ('Phoebe', 0.8803685307502747),
 ('Rachel', 0.8781219720840454),
 ('Monica', 0.8617719411849976),
 ('Mike', 0.6985695958137512),
 ('Ben', 0.6470649838447571),
 ('everybody', 0.6407077312469482),
 ('Emily', 0.6349152326583862),
 ('David', 0.6346350312232971)]

## Visualize word2vec vectors

In [9]:
# visualization
from sklearn.manifold import TSNE
import pandas as pd
import re
import matplotlib.pyplot as plt

In [10]:
vocab = list(model.wv.vocab)
X = model[vocab]

NameError: name 'model' is not defined

In [11]:
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

NameError: name 'X' is not defined

In [12]:
df = pd.DataFrame(X_tsne, index=vocab, columns=['x', 'y'])
df = df[:500]

fig = plt.figure()
fig.set_figwidth(10)
fig.set_figheight(10)

ax = fig.add_subplot(1, 1, 1)
ax.scatter(df['x'], df['y'])

for word, pos in df.iterrows():
    ax.annotate(word, pos)

NameError: name 'X_tsne' is not defined